In [1]:
import sql_py_defs as sql
import matplotlib.pyplot as plt
import numpy as np
import keyring
import os

In [ ]:
pw = keyring.get_password("mysql", "ellaguise")
connection = sql.create_db_connection("localhost", "root", pw, "agn_samp")

In [3]:
q1 = """
SELECT z,lum FROM agn_samp_z_5_area_10;
"""

results = sql.read_query(connection, q1)
z = [results[i][0] for i in range(len(results))]
lum = [results[i][1] for i in range(len(results))]

q2 = """
SELECT tau,sigma FROM agn_DRW_z_5_area_10;
"""

results = sql.read_query(connection, q2)
tau = [results[i][0] for i in range(len(results))]
sigma = [results[i][1] for i in range(len(results))]

alter_drw_table = """
ALTER TABLE agn_DRW_z_5_area_10 
  ADD mean_tau FLOAT NOT NULL,
  ADD std_tau FLOAT NOT NULL,
  ADD mean_sig FLOAT NOT NULL,
  ADD std_sig FLOAT NOT NULL
  ;
 """

#sql.execute_query(connection, alter_drw_table)

In [ ]:
update_drw_table = '''
    UPDATE agn_DRW_z_5_area_10 
    SET mean_tau = %s, std_tau = %s, mean_sig = %s, std_sig = %s
    where id = %s
    '''

get_id = """
SELECT id FROM agn_samp_z_5_area_10 where z = (%s) AND lum = (%s);
"""

import glob
import re


for i in range(len(lum)):
    val = (float(z[i]), float(lum[i]))
    res = sql.read_query(connection, get_id, val)
    idx = res[0][0]
    test_file = "./jav_dats/fchain_lum_{}_z_{}_DRW_cad_DDF.dat".format(round(lum[i],6), round(z[i],6))
    if os.path.isfile(test_file):
        data = np.loadtxt(test_file)
        ln_t = [data[j][1] for j in range(len(data))]
        ln_s = [data[j][0] for j in range(len(data))]
        t = [np.log10(np.power(2.71828182846, ln_t[j])) for j in range(len(data))]
        s = [np.log10(np.power(2.71828182846, ln_s[j])) for j in range(len(data))]
        
        val = [(float(np.mean(t)), float(np.std(t)), float(np.mean(s)), float(np.std(s)), int(idx))]
        sql.execute_list_query(connection, update_drw_table, val)
        
        


Get the distribution of AGN within luminosity and redshift space


In [ ]:
lum_range = [np.floor(min(lum))+(np.ceil(max(lum)-min(lum))/10)*i for i in range(11)]
z_range = [np.floor(min(z))+(np.ceil(max(z)-min(z))/10)*i for i in range(11)]

lum_z_num = np.zeros((10,10))

find_num_agn = """
SELECT COUNT(id) FROM agn_samp_z_5_area_10 where z BETWEEN (%s) AND (%s) and lum BETWEEN (%s) AND (%s);
"""

for i in range(len(lum_range)-1):
    for j in range(len(z_range)-1):
        val = (float(z_range[j]), float(z_range[j+1]), float(lum_range[i]), float(lum_range[i+1]))
        res = sql.read_query(connection, find_num_agn, val)
        lum_z_num[i, j] = res[0][0]

        
plt.imshow(lum_z_num,cmap="plasma")
plt.xlabel('Log Luminosity')
plt.ylabel('Redshift')
ticks = [i-0.5 for i in range(11)]
plt.xticks(ticks, lum_range)
plt.yticks(ticks, z_range)



Find the distribution of AGN that don't successfully recover the value of tau using JAVELIN

In [ ]:
lum_z_tau = np.zeros((10,10))

find_num_agn = """
SELECT COUNT(agn_samp_z_5_area_10.id)
FROM agn_samp_z_5_area_10
INNER JOIN agn_DRW_z_5_area_10 ON agn_samp_z_5_area_10.id=agn_DRW_z_5_area_10.id
WHERE agn_DRW_z_5_area_10.mean_tau = 0 
AND agn_samp_z_5_area_10.z BETWEEN (%s) 
AND (%s) and agn_samp_z_5_area_10.lum BETWEEN (%s) AND (%s);
"""

for i in range(len(lum_range)-1):
    for j in range(len(z_range)-1):
        val = (float(z_range[j]), float(z_range[j+1]), float(lum_range[i]), float(lum_range[i+1]))
        res = sql.read_query(connection, find_num_agn, val)
        if lum_z_num[i,j] > 0:
            lum_z_tau[i, j] = res[0][0]/lum_z_num[i,j]

plt.imshow(lum_z_tau,cmap="plasma")
plt.xlabel('Log Luminosity')
plt.ylabel('Redshift')
ticks = [i-0.5 for i in range(11)]
plt.xticks(ticks, lum_range)
plt.yticks(ticks, z_range)

In [ ]:
lum_z_tau = np.zeros((10,10))

find_num_agn = """
SELECT COUNT(agn_samp_z_5_area_10.id)
FROM agn_samp_z_5_area_10
INNER JOIN agn_DRW_z_5_area_10 ON agn_samp_z_5_area_10.id=agn_DRW_z_5_area_10.id
WHERE agn_DRW_z_5_area_10.tau BETWEEN agn_DRW_z_5_area_10.mean_tau - agn_DRW_z_5_area_10.std_tau AND agn_DRW_z_5_area_10.mean_tau + agn_DRW_z_5_area_10.std_tau
AND agn_samp_z_5_area_10.z BETWEEN (%s) 
AND (%s) and agn_samp_z_5_area_10.lum BETWEEN (%s) AND (%s);
"""

for i in range(len(lum_range)-1):
    for j in range(len(z_range)-1):
        val = (float(z_range[j]), float(z_range[j+1]), float(lum_range[i]), float(lum_range[i+1]))
        res = sql.read_query(connection, find_num_agn, val)
        if lum_z_num[i,j] > 0:
            lum_z_tau[i, j] = res[0][0]/lum_z_num[i,j]

plt.imshow(lum_z_tau,cmap="plasma")
plt.xlabel('Log Luminosity')
plt.ylabel('Redshift')
ticks = [i-0.5 for i in range(11)]
plt.xticks(ticks, lum_range)
plt.yticks(ticks, z_range)

In [ ]:
lum_z_sig = np.zeros((10,10))

find_num_agn = """
SELECT COUNT(agn_samp_z_5_area_10.id)
FROM agn_samp_z_5_area_10
INNER JOIN agn_DRW_z_5_area_10 ON agn_samp_z_5_area_10.id=agn_DRW_z_5_area_10.id
WHERE agn_DRW_z_5_area_10.sigma BETWEEN agn_DRW_z_5_area_10.mean_sig - agn_DRW_z_5_area_10.std_sig AND agn_DRW_z_5_area_10.mean_sig + agn_DRW_z_5_area_10.std_sig
AND agn_samp_z_5_area_10.z BETWEEN (%s) 
AND (%s) and agn_samp_z_5_area_10.lum BETWEEN (%s) AND (%s);
"""

for i in range(len(lum_range)-1):
    for j in range(len(z_range)-1):
        val = (float(z_range[j]), float(z_range[j+1]), float(lum_range[i]), float(lum_range[i+1]))
        res = sql.read_query(connection, find_num_agn, val)
        if lum_z_num[i,j] > 0:
            lum_z_tau[i, j] = res[0][0]/lum_z_num[i,j]

plt.imshow(lum_z_tau,cmap="plasma")
plt.xlabel('Log Luminosity')
plt.ylabel('Redshift')
ticks = [i-0.5 for i in range(11)]
plt.xticks(ticks, lum_range)
plt.yticks(ticks, z_range)
